<a href="https://colab.research.google.com/github/Varunsaistark/gans/blob/main/gan_emnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import torch
from torch import nn
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
import matplotlib.pyplot as plt

In [20]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [50]:
class generator(nn.Module):
  def __init__(self):
    super(generator,self).__init__() 
    self.fc1 = nn.Linear(100,256)
    self.fc2 = nn.Linear(256,512)
    self.fc3 = nn.Linear(512,1024)
    self.fc4 = nn.Linear(1024,1024)
    self.fc5 = nn.Linear(1024,512)
    self.fc6 = nn.Linear(512,784)
    self.drop = nn.Dropout(0.3)
  def forward(self,x):
    out = F.leaky_relu(self.fc1(x),0.2)
    out = self.drop(out)
    out = F.leaky_relu(self.fc2(out),0.2)
    out = self.drop(out)
    out = F.leaky_relu(self.fc3(out),0.2)
    out = self.drop(out)
    out = F.leaky_relu(self.fc4(out),0.2)
    out = self.drop(out)
    out = F.leaky_relu(self.fc5(out),0.2)
    out = self.drop(out)
    out = torch.tanh(self.fc6(out))
    return out


In [51]:
class discrminator(nn.Module):
  def __init__(self):
    super(discrminator,self).__init__()
    self.fc1= nn.Linear(784,1024)
    self.fc2 = nn.Linear(1024,512)
    self.fc3 = nn.Linear(512,512)
    self.fc4 = nn.Linear(512,512)
    self.fc5 = nn.Linear(512,256)
    self.fc6 = nn.Linear(256,1)
    self.drop = nn.Dropout(0.3)
  def forward(self,x):
    out = F.leaky_relu(self.fc1(x),0.2)
    #print(out.shape)
    out = self.drop(out)
    out = F.leaky_relu(self.fc2(out),0.2)
    out = self.drop(out)
    out = F.leaky_relu(self.fc3(out),0.2)
    out = self.drop(out)
    out = F.leaky_relu(self.fc4(out),0.2)
    out = self.drop(out)
    out = F.leaky_relu(self.fc5(out),0.2)
    out = self.drop(out)
    out = F.sigmoid(self.fc6(out))
    return out

In [52]:
gen = generator().to(device)
dis = discrminator().to(device)


In [39]:
import pandas as pd
emdf = pd.read_csv('/content/sample_data/mnist_train_small.csv')
emdf = emdf.drop('6',axis=1)

In [25]:

import numpy as np
y = torch.ones(19999).to(device)
y

tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0')

In [26]:
mu=0.1
sigma =0.001

gen_input = torch.tensor(np.random.normal(mu, sigma, (20000,10))).float().to(device)

In [27]:

gen_data = gen.forward(gen_input)

In [28]:

gen_data.shape

torch.Size([20000, 784])

In [29]:

y_gen = torch.zeros(20000).to(device)

In [30]:
real = torch.tensor(emdf.to_numpy()).float().to(device)
real=real/255
real.shape

torch.Size([19999, 784])

In [31]:

y.shape

torch.Size([19999])

In [53]:
def d_train(fake):
 
  real_x,real_y = dis.forward(real),y.view(19999,-1)
  fake_x,fake_y = dis.forward(fake),y_gen.view(20000,-1)
  loss = nn.BCELoss()
  tr = loss(real_x,real_y)+loss(fake_x,fake_y)
  print("The discriminator loss is {}".format(tr))
  optimizer = optim.Adam(dis.parameters(),lr= 0.0002)
  optimizer.zero_grad()
  tr.backward()
  optimizer.step()


In [54]:
def gen_train():
  gen_input = torch.tensor(np.random.normal(mu, sigma, (20000,100))).float().to(device)
  fake = gen.forward(gen_input)
  f = dis.forward(fake)
  loss = nn.BCELoss()
  y1 = torch.ones(20000).to(device)
  tr = loss(f,y1.view(20000,-1))
  optimizer = optim.Adam(gen.parameters(),lr = 0.0002)
  optimizer.zero_grad()
  tr.backward()
  optimizer.step()
  print("The generator loss is {}".format(tr))


In [55]:
epochs=10000000
k=10
alpha = 0.0002

In [56]:
from google.colab.patches import cv2_imshow
gen_update_frequency =2
for i in range(epochs):
  print("Episode {}".format(i))
  #cv2_imshow(fake.view((28,28)))
  if i%gen_update_frequency ==0:
    gen_train()
    with torch.no_grad():
      inp = torch.tensor(np.random.normal(mu, sigma, (1,100))).float().to(device)
      g = gen.forward(inp)*255
      l = g.view(28,28).cpu().numpy()
      cv2_imshow(l)
  gen_input = torch.tensor(np.random.normal(mu, sigma, (20000,100))).float().to(device)
  fake = gen.forward(gen_input).detach()
  d_train(fake)

  

Episode 0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


The generator loss is 0.6823934316635132


The discriminator loss is 1.3865678310394287
Episode 1
The discriminator loss is 1.3771214485168457
Episode 2
The generator loss is 0.6843734383583069


The discriminator loss is 1.36432683467865
Episode 3
The discriminator loss is 1.344166874885559
Episode 4
The generator loss is 0.6882330775260925


...

The discriminator loss is 0.6880582571029663
Episode 1805
The discriminator loss is 0.27736884355545044
Episode 1806
The generator loss is 2.7420339584350586


The discriminator loss is 1.0927520990371704
Episode 1807
The discriminator loss is 0.29768240451812744
Episode 1808
The generator loss is 3.7068185806274414


The discriminator loss is 0.9899262189865112
Episode 1809
The discriminator loss is 0.3217649757862091
Episode 1810
The generator loss is 2.5477328300476074


The discriminator loss is 1.6765971183776855
Episode 1811
The discriminator loss is 0.39552465081214905
Episode 1812


KeyboardInterrupt: ignored

In [61]:
with torch.no_grad():
  inp = torch.rand(100).to(device)
  g = gen.forward(inp)*255
  l = g.view(28,28).cpu().numpy()
  cv2_imshow(l)
